<a href="https://colab.research.google.com/github/brooklynnae/ENPH353_Lab2/blob/main/ENPH353Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab.patches import cv2_imshow
video = cv2.VideoCapture("/content/drive/MyDrive/raw_video_feed.mp4")


#dimensions of video and (buff) how far down the frame to search for line
w = int(video.get(3))
h = int(video.get(4))
buff = int(4*h/5)
res = (w, h)

#create output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v') #codec
output_file = 'labeled_video_feed.mp4'
out = cv2.VideoWriter(output_file, fourcc, 15.0, res)

#loop through each frame
while True:
  ret, img = video.read()

  #break at the end
  if not ret:
    break

  blur = cv2.GaussianBlur(img, (5, 5), 0) #blur for background smoothing
  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY) #gray for binary mask easier
  ret, mask = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV) #white track, black background
  crop = mask[buff:h, 0:w] #only search below buff
  #cv2_imshow(mask)

  #create blob detection by colour
  params = cv2.SimpleBlobDetector_Params()

  params.filterByColor = True
  params.blobColor = 255
  params.filterByArea = False
  params.filterByCircularity = False
  params.filterByConvexity = False
  params.filterByInertia = False

  #search with detector and report center of track
  detector = cv2.SimpleBlobDetector_create(params)
  keypoints = detector.detect(crop)
  if keypoints:
    x = int(keypoints[0].pt[0])
    y = int(keypoints[0].pt[1]) + buff

  #draw circle on orginial frame and write to output file
  cv2.circle(img, (x,y), 15, (0,0,255), -1)

  out.write(img)

video.release()
out.release()
cv2.destroyAllWindows()
#files.download(output_file)